In [2]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [3]:
preprocessor = mz.models.DUET.get_default_preprocessor(
    filter_mode='df',
    filter_low_freq=2,
    truncated_mode='post',
    truncated_length_left=10,
    truncated_length_right=40,
    ngram_size=3
)

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/95 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.198 seconds.
Prefix dict has been built successfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 95/95 [00:01<00:00, 54.32it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 95/95 [00:00<00:00, 173.14it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 

In [5]:
preprocessor.context

{'embedding_input_dim': 178,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x1d24d8a8828>,
 'ngram_process_unit': <mzcn.preprocessors.units.ngram_letter.NgramLetter at 0x1d24d8a8668>,
 'ngram_vocab_size': 178,
 'ngram_vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x1d23f70d358>,
 'vocab_size': 178,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x1d23f697208>}

In [6]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    callbacks=[triletter_callback]
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    callbacks=[triletter_callback]
)

In [7]:
padding_callback = mz.models.DUET.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=40,
    pad_word_value=0,
    pad_word_mode='pre',
    with_ngram=True
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [8]:
model = mz.models.DUET()

model.params['task'] = ranking_task
model.params['left_length'] = 10
model.params['right_length'] = 40
model.params['lm_filters'] = 100
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 100
model.params['mlp_activation_func'] = 'tanh'

model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['dm_conv_activation_func'] = 'relu'
model.params['dm_filters'] = 100
model.params['dm_kernel_size'] = 3
model.params['dm_right_pool_size'] = 4
model.params['dropout_rate'] = 0.2


model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DUET(
  (lm_conv1d): Conv1d(40, 100, kernel_size=(1,), stride=(1,))
  (lm_mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1000, out_features=100, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
    )
    (2): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): Tanh()
    )
  )
  (lm_linear): Sequential(
    (0): Linear(in_features=100, out_features=1, bias=True)
    (1): ReLU()
  )
  (dm_conv_activation_func): ReLU()
  (dm_conv_left): Conv1d(178, 100, kernel_size=(3,), stride=(1,))
  (dm_mlp_left): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
  )
  (dm_conv1_right): Conv1d(178, 100, kernel_size=(3,), stride=(1,))
  (dm_conv2_right): Conv1d(100, 100, kernel_size=(1,), stride=(1,))
  (dm_mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=900, out_features=100, bias=True)
      (1): 

In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-1 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-2 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-3 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-4 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-5 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-6 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-7 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-8 Loss-1.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-9 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-10 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613

Cost time: 3.4080376625061035s


In [12]:
import gc
gc.collect()

966